# Gradient Bandit algorithm (A k-armed bandit problem)
by __Shivangi Agarwal and Sandeep Banik__ | Reinforcement Learning

All of the pervious methods show some methods of choosing a action based on $Q$ values estimates and some randomness leading to a balance between exploration and exploitation. Here we provide a method to qunatitavely prefer an action over the others. This method is not dependent on the $Q$ value estimates and is a result of the rewards gained in choosing an action $a$. Only relative preference matter, therefore any offset values added to all which not deviate from the current preference. The porbability of choosing an action is given by the Gibbs distribution as,

$$Pr\{A_{t}=a\} = \frac{e^{H_{t}(a)}}{\sum_{i=1}^{n} e^{H_t(i)}} = \pi_{t}(a)$$

Here $A_{t}$ is action chosen at the time $t$, $H_{t}(a)$ is the numerical preference of choosing an action $a$ at the time t, $\pi_{t}(a)$ is the probability distribution of choosing an action $a$ at a given time $t$. Initially we would like no preference over any action leading to $H_{0}(a) = 0$ $\forall a \in \{1,2,..n\}$

## Update rule of $H_t(a)$

The update rule for the numerical preference is given as, 

$ H_{t+1}(a) = H_{t}(a) + \alpha(R_{t} - \bar{R_t})(1 - \pi_t(A_t)) \quad \quad \quad \text{and}$ 

$ H_{t+1}(a) = H_{t}(a) - \alpha(R_{t} - \bar{R_t})\pi_t(A_t) \quad \quad \quad \text{for all  } a \neq A_{t}$

where $\alpha$ is the step size (user defined), $R_{t}$ is the current reward and $\bar{R_t}$ is the average reward upto the time $t$. 
As observed the probability distribution changes over time. A small example is illustrated as follows. Consider a machine with 3 actions or arms. Initially we begin with equal preference i.e., $H_0{a} = 0$ for $a = 1,2,3$. 

<img src="Images/Gradient bandit - probability.png">

The probability distribution keeps changing over time adjusting to the rewards. The average reward $\bar{R_{t}}$ serves as a baseline, without which the gradient bandit would fail to perform. 

When a particular action is chosen the corresponding difference between rewards and average reward ($R_{t} - \bar{R_{t}}$) either create a greater preference (when positive) or lowers it (when negative). Similarly this effects the other actions as well when the difference is ($R_{t} - \bar{R_{t}}$) positive the numerical preference lowers and when difference is nagtive the preference increases. 

## Code and analysis